In [42]:
import numpy as np

In [14]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
import pickle
import pandas as pd
set_id = 3
splits = pickle.load(open(f"../shared/cv1.pkl", "rb"))
dataset = pd.read_csv(f"../shared/df.csv")

for split_name, ids in splits.items():
  int_ids = [int(id) for id in ids]
  dataset.loc[dataset["Id"].isin(int_ids), "split_custom"] = (
      split_name
  )

In [2]:
from datasets import Dataset, DatasetDict
from datasets.features import ClassLabel, Value

# Rename dataset columns
dataset.rename(columns={"Score1": "label", "EssayText": "essay", "Id": "idx"}, inplace=True)
# Drop every other column in the dataset aside from 'label', 'essay', and 'idx'
columns_to_keep = ['label', 'essay', 'idx', "split_custom"]
columns_to_drop = [col for i, col in enumerate(dataset.columns) if col not in columns_to_keep]
dataset_new = dataset.drop(columns=columns_to_drop)

# Create a function to map the split names to the dataset format
def create_dataset_dict(dataset):
    dataset_dict = {}
    for split_name in dataset["split_custom"].unique():
        split_data = dataset[dataset["split_custom"] == split_name]
        dataset_dict[split_name] = Dataset.from_pandas(split_data)

    # Drop the 'split_custom' column from each split in the dataset_dict
    for split_name in dataset_dict:
        dataset_dict[split_name] = dataset_dict[split_name].remove_columns("split_custom")    

    return dataset_dict

# Convert the dataset to DatasetDict format
dataset_dict = create_dataset_dict(dataset_new)

# Create the final DatasetDict object
dataset_dict = DatasetDict(dataset_dict)


In [3]:
model_checkpoint = "bert-base-uncased"
task="set_3"
batch_size = 16

In [4]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [28]:
def preprocess_function(examples):
    return tokenizer(examples["essay"], truncation=True)

In [36]:
encoded_dataset = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/1445 [00:00<?, ? examples/s]

Map:   0%|          | 0/363 [00:00<?, ? examples/s]

Map:   0%|          | 0/406 [00:00<?, ? examples/s]

In [37]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /sailhome/joetey/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tr

In [77]:
from transformers import TrainingArguments
metric_name = "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [78]:
import evaluate
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    kappa_score = cohen_kappa_score(
        labels, predictions, weights="quadratic", sample_weight=None
    )

    accuracy_results = metric.compute(predictions=predictions, references=labels)
    
    return {
        "accuracy": accuracy_results["accuracy"],
        "qwk": kappa_score
    }

# eval_pred = (
#     np.array([[0.1, 0.9, 0.0], [0.8, 0.1, 0.1], [0.2, 0.3, 0.5]]),  # predictions
#     np.array([1, 0, 2])  # labels
# )

# compute_metrics(eval_pred)


In [86]:
validation_key = "valid"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/piech/u/joetey/auto-ed-coder/venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/piech/u/joetey/auto-ed-coder/bert-encoder-fine-tuned-sae/bert-base-uncased-finetuned-set_3 is already a clone of https://huggingface.co/joetey/bert-base-uncased-finetuned-set_3. Make sure you pull the latest changes with `repo.git_pull()`.


In [87]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: essay, __index_level_0__, idx. If essay, __index_level_0__, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/piech/u/joetey/auto-ed-coder/venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1445
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 455


Epoch,Training Loss,Validation Loss,Accuracy,Qwk
1,No log,2.347965,0.724518,0.651925
2,No log,2.161516,0.727273,0.682706
3,No log,2.233676,0.710744,0.673860
4,No log,2.159846,0.721763,0.701905
5,No log,2.186488,0.730028,0.693707


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: essay, __index_level_0__, idx. If essay, __index_level_0__, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 363
  Batch size = 16
Saving model checkpoint to bert-base-uncased-finetuned-set_3/checkpoint-91
Configuration saved in bert-base-uncased-finetuned-set_3/checkpoint-91/config.json
Model weights saved in bert-base-uncased-finetuned-set_3/checkpoint-91/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-set_3/checkpoint-91/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-set_3/checkpoint-91/special_tokens_map.json
tokenizer config file saved in bert-base-uncased-finetuned-set_3/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-set_3/special_tokens_map.jso

TrainOutput(global_step=455, training_loss=0.03982792529431018, metrics={'train_runtime': 112.4553, 'train_samples_per_second': 64.248, 'train_steps_per_second': 4.046, 'total_flos': 367147024196148.0, 'train_loss': 0.03982792529431018, 'epoch': 5.0})

In [88]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: essay, __index_level_0__, idx. If essay, __index_level_0__, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 363
  Batch size = 16


{'eval_loss': 2.186487913131714,
 'eval_accuracy': 0.7300275482093664,
 'eval_qwk': 0.6937067143418073,
 'eval_runtime': 0.5653,
 'eval_samples_per_second': 642.139,
 'eval_steps_per_second': 40.686,
 'epoch': 5.0}

In [89]:
trainer.push_to_hub()

Saving model checkpoint to bert-base-uncased-finetuned-set_3
Configuration saved in bert-base-uncased-finetuned-set_3/config.json
Model weights saved in bert-base-uncased-finetuned-set_3/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-set_3/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-set_3/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.7300275482093664}]}
To https://huggingface.co/joetey/bert-base-uncased-finetuned-set_3
   42d7365..92ce2c6  main -> main

